# **EVALUACION 1 MINERIA DE DATOS 2024 (BIY7121_004V) - STARCRAFT II**

In [2]:
# Importamos librerias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import KNNImputer

## **1. BUSSINESS UNDERSTANDING**

Se nos ha entregado un dataset correspondiente a información de partidas multijugador de Starcraft II, juego RTS (Real Time Strategy) lanzado en 2010 como una secuela del juego lanzado originalmente en 1998. 

In [3]:
# Cargamos el dataset
sc2DF = pd.read_csv("starcraft_duoc.csv", delimiter=';')

## **2. DATA UNDERSTANDING**

In [4]:
# Mostramos los primeros 5 registros
sc2DF.head()

,GameID,LeagueIndex,Age,HoursPerWeek,TotalHours,APM,SelectByHotkeys,AssignToHotkeys,MinimapAttacks,MinimapRightClicks,NumberOfPACs,GapBetweenPACs,ActionLatency,ActionsInPAC,TotalMapExplored,WorkersMade,UniqueUnitsMade,ComplexUnitsMade,ComplexAbilityUsed,MaxTimeStamp
0,52,5,27.0,10.0,3000.0,143.718,0.003515159,0.000219697,0.000109849,0.000392317,0.004849,326.677,408.673,47.508,0.000220,0.0013966,4.71e-05,0,0,127448
1,55,5,23.0,10.0,5000.0,1.292.322,0.003303812,0.000259462,0.000294057,0.000432436,0.004307,329.194,423.454,48.434,0.000381,0.0011935,8.65e-05,0,0.00020757,57812
2,56,4,30.0,10.0,200.0,699.612,0.001101091,0.00033557,0.000293624,0.000461409,0.002926,446.475,753.548,4.043,0.000231,0.00074455,6.29e-05,0,0.00018876,95360
3,57,3,19.0,20.0,400.0,1.076.016,0.001033542,0.000213101,5.33e-05,0.000543409,0.003783,292.203,537.352,49.155,0.000202,0.0004262,7.46e-05,0,0.00038358,93852
4,58,3,32.0,10.0,500.0,1.228.908,0.001136014,0.000327326,0,0.001328558,0.002368,226.885,620.813,9.374,0.000289,0.0011745,7.7e-05,0,1.93e-05,51936


In [5]:
# Los ultimos 5 registros
sc2DF.tail()

,GameID,LeagueIndex,Age,HoursPerWeek,TotalHours,APM,SelectByHotkeys,AssignToHotkeys,MinimapAttacks,MinimapRightClicks,NumberOfPACs,GapBetweenPACs,ActionLatency,ActionsInPAC,TotalMapExplored,WorkersMade,UniqueUnitsMade,ComplexUnitsMade,ComplexAbilityUsed,MaxTimeStamp
3390,10089,8,NaN,NaN,NaN,2.596.296,0.020425314,0.00074296,0.000621163,0.000146156,0.004555,186.059,428.342,62.754,0.000560,0.00087694,6.09e-05,0,0,82104
3391,10090,8,NaN,NaN,NaN,314.67,0.028043136,0.001157179,0.000246208,0.001083317,0.004259,143.023,361.156,71.965,0.000394,0.00078787,9.85e-05,0,0,40616
3392,10092,8,NaN,NaN,NaN,2.994.282,0.028341014,0.000860215,0.000337942,0.000168971,0.004439,124.028,395.156,63.979,0.000292,0.0012596,6.14e-05,0,0,65100
3393,10094,8,NaN,NaN,NaN,3.758.664,0.036435894,0.000594265,0.000204279,0.000779973,0.004346,11.691,348.547,79.615,0.000279,0.00061284,0.000111425,0,0.00063141,53848
3394,10095,8,NaN,NaN,NaN,3.483.576,0.029854921,0.000811174,0.000223772,0.001314661,0.005566,200.537,335.142,63.719,0.000252,0.0015664,6.53e-05,0.000456868,0.00089509,107252


In [24]:
# Una vista a los tipos de datos de cada una de las columnas
sc2DF.dtypes

GameID                  int64
LeagueIndex             int64
Age                   float64
HoursPerWeek          float64
TotalHours            float64
APM                    object
SelectByHotkeys        object
AssignToHotkeys        object
MinimapAttacks         object
MinimapRightClicks     object
NumberOfPACs          float64
GapBetweenPACs         object
ActionLatency          object
ActionsInPAC          float64
TotalMapExplored      float64
WorkersMade            object
UniqueUnitsMade        object
ComplexUnitsMade       object
ComplexAbilityUsed     object
MaxTimeStamp            int64
dtype: object

In [25]:
for column in sc2DF.columns:
    sc2DF[column].astype(float)

ValueError: could not convert string to float: '1.292.322'

### 2.1. **Limpieza y transformación**

#### 2.1.1. **Tratamiento de valores nulos**

In [8]:
sc2DF.isna().sum()

GameID                 0
LeagueIndex            0
Age                   55
HoursPerWeek          56
TotalHours            57
APM                    0
SelectByHotkeys        0
AssignToHotkeys        0
MinimapAttacks         0
MinimapRightClicks     0
NumberOfPACs           0
GapBetweenPACs         0
ActionLatency          0
ActionsInPAC           0
TotalMapExplored       0
WorkersMade            0
UniqueUnitsMade        0
ComplexUnitsMade       0
ComplexAbilityUsed     0
MaxTimeStamp           0
dtype: int64

In [9]:
# Vamos a imputar valores usando KNN
# Crear una instancia de KNNImputer
imputer = KNNImputer(n_neighbors=3, weights='distance') #uniform

# Imputar los valores faltantes
imputed_data = imputer.fit_transform(sc2DF)
sc2DFImputed = pd.DataFrame(imputed_data)

ValueError: could not convert string to float: '1.292.322'

In [ ]:
sc2DF.isna().sum()

: 

#### 2.1.2 **Tratamiento de outliers**

: 

### 2.3. **Análisis y procesamiento**

In [26]:
array = ['1.000.000', 566.32, 56.345, '897.43']

for i in range(len(array)):
    if isinstance(array[i], str):
        # Eliminar los puntos como separadores de miles
        array[i] = float(array[i].replace('.', ''))
    else:
        array[i] = float(array[i])

print(array)

[1000000.0, 566.32, 56.345, 89743.0]
